# Guide: run anything in distributed fashion

In [ ]:
from typing import List

from qiskit import QuantumCircuit, transpile
from qiskit.circuit.random import random_circuit

from quantum_serverless import QuantumServerless, run_qiskit_remote, get, put

In [2]:
serverless = QuantumServerless()

serverless

<QuantumServerless: providers [local], clusters [local]>

In [3]:
circuit = random_circuit(5, 3)

circuit.draw()

┌────────────┐            ┌────────────┐   
q_0: ┤ U1(3.1592) ├─────────■──┤ Rz(4.4834) ├───
     └────────────┘ ┌───┐   │  └────────────┘   
q_1: ──────■────────┤ X ├───┼─────────────────■─
           │       ┌┴───┴┐  │      ┌───┐      │ 
q_2: ──────X───────┤ Tdg ├──┼──────┤ S ├──────┼─
           │       └─────┘  │  ┌───┴───┴────┐ │ 
q_3: ──────X────────────────■──┤ Ry(1.8682) ├─┼─
         ┌───┐            ┌─┴─┐└────────────┘ │ 
q_4: ────┤ X ├────────────┤ X ├───────────────■─
         └───┘            └───┘

In [4]:
# object

with serverless:
    # make any object as remote object
    circuit_reference = put(circuit)

    print("Circuit reference:", circuit_reference)
    
    # to get any object from remote to local call `get` function
    retrieved_object = get(circuit_reference)
    print("Retrieved object\n", retrieved_object)

Circuit reference: ObjectRef(00ffffffffffffffffffffffffffffffffffffff0100000001000000)
Retrieved object
      ┌────────────┐            ┌────────────┐   
q_0: ┤ U1(3.1592) ├─────────■──┤ Rz(4.4834) ├───
     └────────────┘ ┌───┐   │  └────────────┘   
q_1: ──────■────────┤ X ├───┼─────────────────■─
           │       ┌┴───┴┐  │      ┌───┐      │ 
q_2: ──────X───────┤ Tdg ├──┼──────┤ S ├──────┼─
           │       └─────┘  │  ┌───┴───┴────┐ │ 
q_3: ──────X────────────────■──┤ Ry(1.8682) ├─┼─
         ┌───┐            ┌─┴─┐└────────────┘ │ 
q_4: ────┤ X ├────────────┤ X ├───────────────■─
         └───┘            └───┘                 


/opt/anaconda3/envs/quantum-serverless/lib/python3.10/site-packages/ray/_private/worker.py:976: UserWarning: len(ctx) is deprecated. Use len(ctx.address_info) instead.
  warnings.warn("len(ctx) is deprecated. Use len(ctx.address_info) instead.")


In [5]:
circuit_batches = [
    [random_circuit(5, 3) for _ in range(2)]
    for n_batch in range(3)
]
circuit_batches

[[<qiskit.circuit.quantumcircuit.QuantumCircuit at 0x7fd840eb84f0>,

In [6]:
# task

# in order to make any function a remote/parallel function annotate it with `remote` decorator

@run_qiskit_remote()
def remote_transpile(circuits: List[QuantumCircuit]):
    return transpile(circuits)

with serverless.context():
    # to call this function by arring `remote` to function call `<function_name>.remote`
    remote_function_reference = remote_transpile(circuit_batches[0])
    print("Function reference:", remote_function_reference)
    
    # to get function results we again using `get`
    result = get(remote_function_reference)
    print("Function result:", result)

Function reference: ObjectRef(c8ef45ccd0112571ffffffffffffffffffffffff0100000001000000)
Function result: [<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7fd840eb8910>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7fd860585f90>]


In [7]:
# we can run multiple remote function in parallel

with serverless:
    # to call this function by arring `remote` to function call `<function_name>.remote`
    remote_function_references = [
        remote_transpile(circuit_batch)
        for circuit_batch in circuit_batches
    ]
    print("Functions references:", remote_function_references)
    
    # to get function results we again using `get`
    result = get(remote_function_references)
    print("Functions resulst:", result)

Functions references: [ObjectRef(16310a0f0a45af5cffffffffffffffffffffffff0100000001000000), ObjectRef(c2668a65bda616c1ffffffffffffffffffffffff0100000001000000), ObjectRef(32d950ec0ccf9d2affffffffffffffffffffffff0100000001000000)]
Functions resulst: [[<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7fd8605868f0>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7fd860586d70>], [<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7fd860586200>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7fd860585d20>], [<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7fd840eb85b0>, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7fd840ec2b00>]]
